In [9]:
from pathlib import Path
import sys, os

PROJECT_ROOT = Path.cwd().resolve()
SRC = PROJECT_ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
# from dotenv import load_dotenv
# load_dotenv(PROJECT_ROOT / "assets" / "config.env")

%load_ext autoreload
%autoreload 2

import nltk
nltk.download('punkt')       
nltk.download('stopwords')   
nltk.download('averaged_perceptron_tagger')  


import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from itables import init_notebook_mode, show
import warnings
from IPython.display import display, HTML


def display_table_with_html(df, title=""):
    """Display pandas DataFrame as styled HTML table"""
    display(HTML(f"<h3>{title}</h3>"))
    html_str = df.to_html(classes='table table-striped table-hover', border=0)
    display(HTML(html_str))

print("Environment ready")

# Load dataset
DATA_PATH = Path("../data/exports/sec_filings_small_full.parquet")
df = pl.read_parquet(DATA_PATH)

print(f"Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.estimated_size('mb'):.1f} MB")

## ------------------------------------------------------------------------------------------------
## Display functions for sentences with HTML/CSS styling


from IPython.display import HTML, display
import textwrap

def display_full_sentences(df, max_rows=50, wrap_length=120, table_height="600px"):
    """
    Display dataframe with wrapped sentences in a scrollable table.
    
    Args:
        df: Polars DataFrame to display
        max_rows: Maximum rows to display (default 50)
        wrap_length: Characters per line for sentence wrapping (default 120)
        table_height: Height of scrollable area (default "600px")
    """
    
    # Define which columns to display and their widths
    display_cols = ["name", "year", "section", "sentenceID", "sentence", "likely_kpi", "has_numbers"]
    
    # Create HTML with custom CSS
    ##  ## 'Courier New', monospace;

    html_content = f"""
    <style>
        .sentence-table {{
            border-collapse: collapse;
            width: 100%;
            font-size: 12px;
            font-family: 'Consolas', monospace;
            background-color: #1e1e1e;          /* Dark background */
            color: #d4d4d4;                     /* Light gray text */
        }}
        .sentence-table th {{
            background-color: #2d2d30;          /* Dark gray header */
            color: #4EC9B0;                     /* Teal text for headers */

            padding: 8px;
            text-align: left;
            position: sticky;
            top: 0;
            z-index: 10;
        }}
        .sentence-table td {{
            border: 1px solid #ddd;
            padding: 8px;
            vertical-align: top;
        }}
        .sentence-table tr: {{
            background-color: #1e1e1e;
        }}
        .sentence-table tr:hover {{
            background-color: #2a2d2e;          /* Subtle hover effect */
            color: #ffffff;                     /* Brighter text on hover */
        }}
        .sentence-cell {{
            white-space: pre-wrap;
            word-wrap: break-word;
            max-width: 600px;
            line-height: 1.5;                    /* CHANGE LINE SPACING HERE */
            color: #d4d4d4;                     /* Light gray for sentences */
            font-weight: 400;                   /* CHANGE FONT WEIGHT HERE */
        }}
        .table-container {{
            height: {table_height};
            overflow: auto;
            border: 2px solid #4CAF50;
            position: relative;
        }}
        .kpi-true {{
            background-color: #3a3a00 !important;  /* Very dark yellow/gold */
            color: #ffeb3b !important;             /* Light yellow text for KPI rows */
        }}
        .has-numbers {{
            font-weight: bold;
            color: #569cd6;                        /* Light blue for numbers */
        }}
        .metadata {{
            color: #808080;                        /* Gray for metadata */
            font-size: 11px;
            background-color: #1e1e1e;
            padding: 5px;
        }}

        /* Custom scrollbar for better dark mode */
        .table-container::-webkit-scrollbar {{
            width: 12px;
            height: 12px;
        }}
        .table-container::-webkit-scrollbar-track {{
            background: #2d2d30;
        }}
        .table-container::-webkit-scrollbar-thumb {{
            background: #555;
            border-radius: 6px;
        }}
        .table-container::-webkit-scrollbar-thumb:hover {{
            background: #007ACC;
        }}

    </style>
    
    <div class="table-container">
        <table class="sentence-table">
            <thead>
                <tr>
    """
    
    # Add headers
    for col in display_cols:
        width = "40%" if col == "sentence" else "auto"
        html_content += f"<th style='width: {width}'>{col}</th>"
    html_content += "</tr></thead><tbody>"
    
    # Add data rows
    for i, row in enumerate(df.head(max_rows).iter_rows(named=True)):
        if i >= max_rows:
            break
            
        kpi_class = "kpi-true" if row.get('likely_kpi', False) else ""
        html_content += f"<tr class='{kpi_class}'>"
        
        for col in display_cols:
            value = row.get(col, "")
            
            if col == "sentence":
                # Wrap long sentences
                if isinstance(value, str):
                    wrapped = textwrap.fill(value, width=wrap_length)
                    html_content += f"<td class='sentence-cell'>{wrapped}</td>"
                else:
                    html_content += f"<td>{value}</td>"
            elif col in ["likely_kpi", "has_numbers"]:
                # Format boolean values
                display_val = "✓" if value else "-"
                extra_class = "has-numbers" if col == "has_numbers" and value else ""
                html_content += f"<td class='{extra_class}'>{display_val}</td>"
            elif col == "section":
                # Add section names
                section_names = {0: "Bus", 1: "Risk", 8: "MD&A", 9: "FinStmt", 10: "Notes", 11: "MktRisk"}
                section_name = section_names.get(value, str(value))
                html_content += f"<td>{value} ({section_name})</td>"
            else:
                html_content += f"<td>{value}</td>"
        
        html_content += "</tr>"
    
    html_content += f"""
            </tbody>
        </table>
    </div>
    <div class='metadata'>
        Displaying {min(max_rows, len(df))} of {len(df)} total rows | 
        Wrap: {wrap_length} chars | 
        Scroll both directions enabled
    </div>
    """
    
    display(HTML(html_content))
    
#: Simple wrap function for single column display
def display_sentences_simple(df, sentence_col="sentence", wrap_at=150):
    """
    Simpler display focusing just on sentences with metadata
    """
    html = "<div style='height: 600px; overflow-y: scroll; border: 1px solid #ccc; padding: 10px;'>"
    
    for row in df.head(100).iter_rows(named=True):
        wrapped_sentence = textwrap.fill(row[sentence_col], width=wrap_at)
        
        html += f"""
        <div style='margin-bottom: 15px; padding: 10px; background: #f9f9f9; border-left: 3px solid #4CAF50;'>
            <div style='color: #666; font-size: 11px; margin-bottom: 5px;'>
                <strong>{row['name']}</strong> | Year: {row.get('year', 'N/A')} | 
                Section: {row.get('section', 'N/A')} | ID: {row.get('sentenceID', 'N/A')}
            </div>
            <div style='font-family: monospace; font-size: 12px; white-space: pre-wrap;'>
{wrapped_sentence}
            </div>
        </div>
        """
    
    html += "</div>"
    display(HTML(html))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Environment ready
Dataset loaded: 200,000 rows × 19 columns
Memory usage: 144.1 MB


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\joems\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joems\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\joems\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [25]:
import polars as pl

df = pl.read_parquet("../data/exports/sec_filings_small_full.parquet")

KPI_SECTIONS = [
    0,   # Business - revenue streams, market share
    1,   # Risk Factors - concentration risks, sensitivities  
    8,   # MD&A - YoY comparisons, margin analysis
    9,   # Financial Statements - raw numbers
    10,  # Notes - detailed breakdowns, segments
    11,  # Market Risk - derivatives, hedging
]

TARGET_YEARS = ['2019', '2020']

kpi_sentences = (
    df.lazy()
    .filter(pl.col("reportDate").str.slice(0, 4).is_in(TARGET_YEARS))
    .filter(pl.col("section").is_in(KPI_SECTIONS))

    # .with_columns([ pl.col("section").str.extract(r"^(\d+)", 1).cast(pl.Int32).alias("section_num") ])
    .with_columns([
        pl.col("sentence").str.len_chars().alias("sentence_length"),
        pl.col("reportDate").str.slice(0, 4).alias("year"),

        pl.when(
            pl.col("sentence").str.contains(r"(?i)(revenue|sales|margin|cost|expense|income|earnings|EBITDA|cash flow)")
        ).then(True).otherwise(False).alias("likely_kpi"),
    
        pl.when(pl.col("sentence").str.contains(r"\$?\d+\.?\d*\s*(million|billion|M|B|%)"))
        .then(True).otherwise(False).alias("has_numbers"),
    ])
    .with_columns(
        pl.count().over("name").alias("company_total_sentences")
    )
    .collect()
)

### top 15 filter on density bad idea -- dont do now for eda.


# top companies
# final_kpi_df = (
#     kpi_sentences
#     .filter(pl.col("name").is_in(top_companies))
#     .sort(["name", "year", "section", "sentenceID"])
# )

# final_kpi_df 


## just a basic long filter and basic section filter for 8 9 10. 
final_kpi_df = (
    kpi_sentences
    .filter( pl.col("likely_kpi") | pl.col("has_numbers") )
    .filter(pl.col("section").is_in([8, 9, 10, 11, 0, 1]))
    .filter( pl.col("sentence_length") > 100 )
    .sort(["year", "section", "name"]) 
    .head(400)  
)



C:\Users\joems\AppData\Local\Temp\ipykernel_29188\3333714674.py:34: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().over("name").alias("company_total_sentences")


In [ ]:

# cik, sentence, section, labels, filingDate, name, docID, sentenceID, sentence_count, ticker, entity_type, sic, stateOfIncorporation, sentence_length	year	likely_kpi	has_numbers	company_total_sentences

display_full_sentences(
    final_kpi_df.select([ "cik", "section", "labels", "filingDate", "name", "docID", "sentence_length", 
                         "sentenceID", "sentence",
                        #  "ticker", "entity_type", "sic", "stateOfIncorporation",  
                         "year", "likely_kpi", "has_numbers", "company_total_sentences" ]),
    max_rows=100,        
    wrap_length=120,     
    table_height="500px" 
)


## 

name,year,section,sentenceID,sentence,likely_kpi,has_numbers
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_68,"Sales to Government and Defense Customers Sales to global government and defense customers (including sales to branches, agencies, and departments of the U.S. government) were $677.9 million (33.0% of consolidated sales), $428.9 million (24.5% of consolidated sales) and $438.8 million (27.6% of consolidated sales) in fiscal 2019, 2018 and 2017, respectively.",✓,✓
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_70,Sales to government and defense customers are reported in each of our reportable segments (See Note 13 of Notes to Consolidated Financial Statements).,✓,-
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_16,"Over the past three years, we have returned $74.2 million to shareholders through common stock repurchases of $43.2 million and dividends of $31.0 million.",-,✓
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_37,"For cost plus fixed fee contracts, we typically receive reimbursement of our costs, to the extent the costs are allowable under contractual and regulatory provisions, in addition to receiving a fixed fee.",✓,-
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_19,"Business activities in this segment are primarily conducted through AAR Supply Chain, Inc.; AAR Government Services, Inc.; AAR Aircraft & Engine Sales & Leasing, Inc.; AAR Aircraft Services, Inc.; AAR Allen Services, Inc.; AAR Landing Gear LLC; AAR Airlift Group, Inc.; and AAR International, Inc. We sell and lease a wide variety of new, overhauled and repaired engine and airframe parts and components and aircraft to our commercial aviation and government/defense customers.",✓,-
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_46,"The Expeditionary Services segment accounted for approximately 5% of our sales in fiscal 2019, 2018, and 2017. Business activities in this segment are primarily conducted through AAR Manufacturing, Inc. and Brown International Corporation.",✓,✓
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_51,Sales in this segment are generally made to customers pursuant to standard commercial purchase orders and contracts.,✓,-
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_52,"Government sales are generally made under standard types of government contracts, which can include firm fixed-price contracts, cost plus fixed fee contracts, and time-and-materials contracts.",✓,-
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_74,"Our government contracts are also often subject to termination for convenience by the customer; in the event of such a termination, we are contractually entitled to recover all allowable costs incurred by us through the date of termination.",✓,-
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_54,"Raw Materials Although we generated approximately 55% of our fiscal 2019 sales from the sale of products, our businesses are generally engaged in limited manufacturing activities and have minimal exposure to fluctuations in both the availability and pricing of raw materials.",✓,✓


In [ ]:
s9_kpi_df = (
    kpi_sentences
    .filter( pl.col("likely_kpi") | pl.col("has_numbers") )
    .filter(
        pl.col("section").is_in([9, 10])  
    )

    # .filter( pl.col("sentence_length") > 100 )
    .sort(["year", "section", "name"]) 
    .head(400)  
)


display_full_sentences(
    s9_kpi_df.select([ "cik", "section", "labels", "filingDate", "name", "docID", 
                         "sentence_length", "sentenceID", "sentence",
                         # "ticker", "entity_type", "sic", "stateOfIncorporation",  
                         "year", "likely_kpi", "has_numbers", "company_total_sentences" ]),
    max_rows=100,        
    wrap_length=120,     
    table_height="500px" 
)

name,year,section,sentenceID,sentence,likely_kpi,has_numbers
ABBOTT LABORATORIES,2019,9 (FinStmt),0000001800_10-K_2019_section_7A_31,"At December 31, 2016, the value of this short-term debt was $454 million, and changes in the fair value of the debt up through the date of repayment due to changes in exchange rates were recorded in Accumulated other comprehensive income (loss), net of tax.",✓,✓
"ADAMS RESOURCES & ENERGY, INC.",2019,9 (FinStmt),0000002178_10-K_2019_section_7A_13,"A hypothetical ten percent additional adverse change in average crude oil prices, assuming no changes in volume levels, would have reduced earnings by approximately $2.7 million and $2.3 million for the years ended December 31, 2019 and 2018, respectively.",✓,✓
"Air Products & Chemicals, Inc.",2019,9 (FinStmt),0000002969_10-K_2019_section_7A_32,"We estimate that a 10% reduction in either the Chinese Renminbi or the Euro versus the U.S. Dollar would lower our annual operating income by approximately $40 and $25, respectively.",✓,✓
CECO ENVIRONMENTAL CORP,2019,9 (FinStmt),0000003197_10-K_2019_section_7A_6,"Market risk was estimated as the potential decrease (increase) in future earnings and cash flows resulting from a hypothetical 10% increase (decrease) in the Company’s estimated weighted average borrowing rate at December 31, 2019.",✓,✓
AAR CORP,2019,10 (Notes),0000001750_10-K_2019_section_8_313,"The income tax rate reduction in the Tax Reform Act was effective January 1, 2018 which resulted in a blended federal statutory tax rate of 29.2% in fiscal 2018.",✓,✓
AAR CORP,2019,10 (Notes),0000001750_10-K_2019_section_8_324,"Income tax receivable was $1.1 million and $1.0 million at May 31, 2019 and 2018, respectively, and was included in Other current assets on the Consolidated Balance Sheet.",✓,✓
AAR CORP,2019,10 (Notes),0000001750_10-K_2019_section_8_54,"For this change, we recognized an increase of $1.3 million to the opening balance of retained earnings as of June 1, 2018.",✓,✓
AAR CORP,2019,10 (Notes),0000001750_10-K_2019_section_8_30,"We adopted this ASU on June 1, 2017 and recognized excess tax benefits of $2.7 million and $2.9 million as an income tax benefit in fiscal 2019 and 2018, respectively.",✓,✓
AAR CORP,2019,10 (Notes),0000001750_10-K_2019_section_8_50,"For this change, we recognized a decrease of $22.1 million to the opening balance of retained earnings as of June 1, 2018.",✓,✓
AAR CORP,2019,10 (Notes),0000001750_10-K_2019_section_8_45,"We recognized the cumulative effect of initially applying ASC 606 as a decrease of $20.4 million to the opening balance of retained earnings as of June 1, 2018.",✓,✓


In [ ]:
s0_kpi_df = (
    kpi_sentences
    .filter( pl.col("likely_kpi") | pl.col("has_numbers") )
    .filter(
        pl.col("section").is_in([0])  
    )

    # .filter( pl.col("sentence_length") > 100 )
    .sort(["year", "section", "name"]) 
    .head(400)  
)


display_full_sentences(
    s0_kpi_df.select([ "cik", "section", "labels", "filingDate", "name", "docID", 
                         "sentence_length", "sentenceID", "sentence",
                        #  "ticker", "entity_type", "sic", "stateOfIncorporation",  
                         "year", "likely_kpi", "has_numbers", "company_total_sentences" ]),
    max_rows=100,        
    wrap_length=120,     
    table_height="500px" 
)

name,year,section,sentenceID,sentence,likely_kpi,has_numbers
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_17,"Our cash on hand plus unused capacities on our Revolving Credit Facility and accounts receivable financing program was $465 million at May 31, 2019. Business Segments Aviation Services The Aviation Services segment provides aftermarket support and services for the commercial aviation and government and defense markets and accounted for approximately 95% of our sales in fiscal 2019, 2018, and 2017.",✓,✓
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_93,"At May 31, 2019, backlog was approximately $1.5 billion and we expect that approximately 40% of this backlog will be recognized as revenue over the next 12 months, with the majority of the remaining balance recognized as revenue over the next three years.",✓,✓
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_69,"Sales to branches, agencies, and departments of the U.S. government and their contractors were $546.2 million (26.6% of consolidated sales), $304.3 million (17.4% of consolidated sales) and $321.5 million (20.2% of consolidated sales) in fiscal 2019, 2018, and 2017, respectively.",✓,✓
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_68,"Sales to Government and Defense Customers Sales to global government and defense customers (including sales to branches, agencies, and departments of the U.S. government) were $677.9 million (33.0% of consolidated sales), $428.9 million (24.5% of consolidated sales) and $438.8 million (27.6% of consolidated sales) in fiscal 2019, 2018 and 2017, respectively.",✓,✓
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_54,"Raw Materials Although we generated approximately 55% of our fiscal 2019 sales from the sale of products, our businesses are generally engaged in limited manufacturing activities and have minimal exposure to fluctuations in both the availability and pricing of raw materials.",✓,✓
AAR CORP,2019,0 (Bus),0000001750_10-K_2019_section_1_46,"The Expeditionary Services segment accounted for approximately 5% of our sales in fiscal 2019, 2018, and 2017. Business activities in this segment are primarily conducted through AAR Manufacturing, Inc. and Brown International Corporation.",✓,✓
ACME UNITED CORP,2019,0 (Bus),0000002098_10-K_2019_section_1_6,Total net sales in 2019 were $142.5 million.,✓,✓
ACME UNITED CORP,2019,0 (Bus),0000002098_10-K_2019_section_1_58,The Company had one customer in 2019 and 2018 that individually exceeded 10% of consolidated net sales.,✓,✓
ACME UNITED CORP,2019,0 (Bus),0000002098_10-K_2019_section_1_59,"Net sales to this one customer were approximately 17% and 16% of consolidated net sales in 2019 and 2018, respectively.",✓,✓
ACME UNITED CORP,2019,0 (Bus),0000002098_10-K_2019_section_1_11,"Recent accomplishments and initiatives The Company’s key business accomplishments and initiatives include the following elements: • Leader in innovative safety solutions and cutting technology; • Ten years of consecutive sales growth averaging 9%; • Innovation rate approaching 30%; • Premier and diversified customer base; • Successful history of acquisitions; • Diversified distribution network; strong growth in e-commerce channel; and • Reduced debt by 25% in 2019 Acquisitions On January 7, 2020, the Company purchased the assets of First Aid Central, located in Laval, Quebec, Canada.",✓,✓


In [ ]:
s0_kpi_df = (
    kpi_sentences
    .filter( pl.col("likely_kpi") | pl.col("has_numbers") )
    .filter(
        pl.col("section").is_in([1])  
    )

    # .filter( pl.col("sentence_length") > 100 )
    .sort(["year", "section", "name"]) 
    .head(400)  
)


display_full_sentences(
    s0_kpi_df.select([ "cik", "section", "labels", "filingDate", "name", "docID", "sentence_length", "sentenceID", "sentence",
                        #  "ticker", "entity_type", "sic", "stateOfIncorporation",  
                         "year", "likely_kpi", "has_numbers", "company_total_sentences" ]),
    max_rows=100,        
    wrap_length=120,     
    table_height="500px" 
)

name,year,section,sentenceID,sentence,likely_kpi,has_numbers
AAR CORP,2019,1 (Risk),0000001750_10-K_2019_section_1A_14,"Our sales to branches, agencies and departments of the U.S. government and their contractors were $546.2 million (26.6% of consolidated sales) in fiscal 2019 (See Note 13 of Notes to Consolidated Financial Statements).",✓,✓
ACME UNITED CORP,2019,1 (Risk),0000002098_10-K_2019_section_1A_53,This latest round affects approximately 10% of the Company’s product purchases and will increase our costs of procurement.,✓,✓
ACME UNITED CORP,2019,1 (Risk),0000002098_10-K_2019_section_1A_70,The Company had one customer in 2019 and 2018 that individually exceeded 10% of consolidated net sales.,✓,✓
ACME UNITED CORP,2019,1 (Risk),0000002098_10-K_2019_section_1A_71,"Net sales to this one customer were approximately 17% and 16% of consolidated net sales in 2019 and 2018, respectively.",✓,✓
ACME UNITED CORP,2019,1 (Risk),0000002098_10-K_2019_section_1A_80,Our e-commerce business constituted approximately 12% of our net sales in 2019 and has been our fastest growing distribution channel over the last several years.,✓,✓
"Air Products & Chemicals, Inc.",2019,1 (Risk),0000002969_10-K_2019_section_1A_17,"In fiscal year 2019, over 60% of our sales were derived from customers outside the United States and many of our operations, suppliers, and employees are located outside the United States.",✓,✓
BK Technologies Corp,2019,1 (Risk),0000002186_10-K_2019_section_1A_30,"For the year ended December 31, 2019, approximately 49.1% of our sales were to agencies and departments of the U.S. Government.",✓,✓
BK Technologies Corp,2019,1 (Risk),0000002186_10-K_2019_section_1A_222,"Future sales of shares of our common stock may negatively affect our stock price and impair our ability to raise equity capital Approximately 6.3 million (49.9%) of our shares of outstanding common stock as of December 31, 2019 were owned by certain of our executive officers and directors and other affiliates, and may be resold publicly at any time, subject to the volume and other restrictions under Rule 144 of the Securities Act of 1933, as amended.",✓,✓
CECO ENVIRONMENTAL CORP,2019,1 (Risk),0000003197_10-K_2019_section_1A_150,"Given that approximately 30% of our 2019 revenues are outside the United States, we are subject to the impact of fluctuations in foreign currency exchange rates.",✓,✓
CECO ENVIRONMENTAL CORP,2019,1 (Risk),0000003197_10-K_2019_section_1A_96,"For the year ended December 31, 2019, approximately 30% of our total revenue was derived from products or services ultimately delivered or provided to end users outside the United States.",✓,✓
